In [63]:
import xml.etree.ElementTree as ET
from nltk.stem import PorterStemmer
import re
from collections import Counter, defaultdict
import numpy as np
import math

In [64]:
f = open('stopwords.txt', "r", encoding="utf-8-sig")
STwords = [word.rstrip() for word in f.readlines()]
stemmer = PorterStemmer()

In [65]:
tree = ET.parse('trec.sample.xml')
root = tree.getroot()

## Create inv index

In [66]:
def create_invindex(file):
    
    tree = ET.parse(file)
    root = tree.getroot()

    inv_index = defaultdict(list)
    docs = []

    for idx,child in enumerate(root.findall('.//DOC')):#iterate through all docs (DOC) each containing HEADLINE DOC TEXT
        headline = child.find('.//HEADLINE').text #get the headline of the nth doc 
        docnum = child.find('.//DOCNO').text #get the docno of the nth doc 
        text = child.find('.//TEXT').text #get the text of the nth doc 
        combined = headline.strip() + " " + text #combination of text+headline
        tokenized = re.findall(r'\d+\.\d+(?:bn|m)?|\w+', combined)#applied tokenization using regex on the combined text+headline
        docs.append(docnum)# get the ids 
        tokenized = [stemmer.stem(word) for word in tokenized if word.lower() not in STwords]#applied stopwords + stemming on tokenized 
        for pos, word in enumerate(tokenized):
            if word in inv_index: #check if the word is in the inverted index first
                if docnum in inv_index[word][1]:#check if the document is in the inverted index of the word
                    inv_index[word][1][docnum].append(pos) # If the the word already ocurred in the docnum, append the position of the new occurrence.
                else:
                    inv_index[word][1][docnum] = [pos] # Else, leave position list untouched.

            else:#if it is not in the inv index then add it
                inv_index[word].append('') # If word is not in inv_index, create key and set frequency to 1.
                inv_index[word].append({}) # Declare posting list for new word.   
                inv_index[word][1][docnum] = [pos] # For each word, add document ID : [pos] pair.

            inv_index[word][0] = len(inv_index[word][1].keys()) # If the word already exists in the inv index, increase the frequency of word.
            #inv_index[word][0] -> frequency part >8
            #inv_index[word][1] -> posting list
    return inv_index, docs

In [67]:
inv_index, docs = create_invindex('trec.sample.xml')

In [68]:
inv_index["tax"]
'''
'jet': [8,
              {
               '1': [5, 22, 39, 58, 84, 103],
               '130': [104, 147, 233],
               '138': [563],
               '188': [34],
               '3556': [12],
               '3726': [40, 49, 58, 93, 380, 442, 523, 945],
               '3808': [63],
               '3923': [97, 326]
               }
        ],
'''

"\n'jet': [8,\n              {\n               '1': [5, 22, 39, 58, 84, 103],\n               '130': [104, 147, 233],\n               '138': [563],\n               '188': [34],\n               '3556': [12],\n               '3726': [40, 49, 58, 93, 380, 442, 523, 945],\n               '3808': [63],\n               '3923': [97, 326]\n               }\n        ],\n"

## Boolean search parser

In [69]:
def get_word(inv_index,word):
    if word in inv_index.keys():
        return inv_index[word]
    else:
        return []
get_word(inv_index,'jet')
# . keys()> docs .values() positions 

[8,
 {'1': [5, 22, 39, 58, 84, 103],
  '130': [104, 147, 233],
  '138': [563],
  '188': [34],
  '3556': [12],
  '3726': [40, 49, 58, 93, 380, 442, 523, 945],
  '3808': [63],
  '3923': [97, 326]}]

In [70]:
def proximity_search(inv_index,word1,word2,distance=1):
    #a = inv_index[word1][1]
    a = get_word(inv_index,word1)[1]
    #b = inv_index[word2][1]
    b = get_word(inv_index,word2)[1]
    match = a.keys() & b.keys()
    newdict = {k:(a[k],b[k]) for k in match}# temp dictionary to store {documentID: ([positions of word a], [positions of word b])},
    documents = set()
    for key,val in newdict.items():
        for i in range(len(val)-1):#i is always 0 that works on list of words A
            for j in range(len(val[i])):# j works on each element of list of words A (length of list A)
                temp = val[i][j]
                for k in range(len(val[i+1])):# k works on each element of list of words B(length of list B)
                    temp2 = val[i+1][k]
                    if distance == 1:
                        if temp2 - temp == distance:
                            documents.add(key)
                    else:
                        if abs(temp2 - temp) <= distance:
                            documents.add(key)
                        

    documents = sorted(list(map(int,documents)))
    return documents

proximity_search(inv_index, 'incom', 'tax', 10)


[65,
 92,
 163,
 282,
 361,
 3387,
 3405,
 3441,
 3449,
 3490,
 3533,
 3562,
 3599,
 3699,
 3706,
 3708,
 3734,
 3817,
 3818]

In [71]:
def query_parser(query,inv_index):
    print(query)
    query = query.lower()#lowercase query
    
    if query.find(" ") == -1 and query.find('#') == -1: #if query has no spaces and no #, means we only working on one word, then return the document ids
        word = stemmer.stem(query)
        docs = get_word(inv_index,word)[1].keys()
        return sorted(map(int,docs))
    
    if query.find('"') != -1:
        tokenized_query = re.findall(r'"(.+?)"',query)[0]#regex to return the first [0] phrase queryprocessed "your query here"
        tokenized_query = tokenized_query.split()# apply tokenization
        tokenized_query = [stemmer.stem(word) for word in tokenized_query]#apply stemming
        temp1 = tokenized_query #temp1 contains two words a and b
        result = proximity_search(inv_index,tokenized_query[0],tokenized_query[1]) #proximity search with distance=1 means phrase search.
        if query.find(' and ') != -1:
            if query.find('not') != -1:
                if query.count('"') == 4:
                    tokenized_query = re.findall(r'"(.+?)"',query)[1]
                    tokenized_query = tokenized_query.split()
                    tokenized_query = [stemmer.stem(word) for word in tokenized_query]
                    notword = re.findall(r'not\s"(.+?)"',query)[0].split()
                    notword = [stemmer.stem(word) for word in notword]
                    if notword == temp1:
                        temp = proximity_search(inv_index,tokenized_query[0],tokenized_query[1])
                        return sorted(set(temp).difference(set(result)))
                    else:
                        temp = proximity_search(inv_index,tokenized_query[0],tokenized_query[1])
                        return sorted(set(result).difference(set(temp)))
                
                notword = re.findall(r'not\s(.+?)(?:$|\s)',query)[0]
                w = stemmer.stem(re.sub(r'"(.*)"|and|not|\s','',query))
                w = get_word(inv_index, w)[1].keys()
                if notword.find('"') != -1:
                    return sorted(set(map(int,w)).difference(set(result)))
                else:
                    return sorted(set(result).difference(set(map(int,w))))
            
            else:
                if query.count('"') == 4: #if the query contains two phrases
                    tokenized_query = re.findall(r'"(.+?)"',query)[1]
                    tokenized_query = tokenized_query.split()
                    tokenized_query = [stemmer.stem(word) for word in tokenized_query]
                    temp = proximity_search(inv_index,tokenized_query[0],tokenized_query[1])
                    return sorted(set(map(int,temp)).intersection(set(result)))
                    
                w = stemmer.stem(re.sub(r'"(.*)"|and|\s','',query))#remove everything except for the word after and
                w = get_word(inv_index, w)[1].keys()
                return sorted(set(map(int,w)).intersection(set(result)))
        
        elif query.find(' or ') != -1:
            if query.count('"') == 4:
                tokenized_query = re.findall(r'"(.+?)"',query)[1]
                tokenized_query = tokenized_query.split()
                tokenized_query = [stemmer.stem(word) for word in tokenized_query]
                temp = proximity_search(inv_index,tokenized_query[0],tokenized_query[1])
                return sorted(set(result).union(temp))
            else:
                w = stemmer.stem(re.sub(r'"(.*)"|or|\s','',query))
                w = get_word(inv_index, w)[1].keys()
                return sorted(set(result).union(set(map(int,w))))
        
        else:
            return result
        
    if query.find('#') != -1:
        distance = int(re.findall(r'\d+', query)[0])#get the number in the query > the distance
        tokenized_query = re.findall(r'\((.+)\)', query)[0]#regex to return anything inside parantheses
        tokenized_query = tokenized_query.split(',')#tokenization
        tokenized_query = [stemmer.stem(word.strip()) for word in tokenized_query]#stemming
        return proximity_search(inv_index,tokenized_query[0],tokenized_query[1],distance=distance)#proximity search with specific distance
    
    if query.find(' and ') != -1:
        if query.find('not') != -1:#incase of and not
            notword = re.findall(r'not\s(.+?)(?:$|\s)',query)[0]#contains the word "NOT word"
            w = re.sub(notword, '', query) #remove 'not word' from query and put it in w
            w = stemmer.stem(re.sub(r'and|not', '', w).strip())# remove 'and' & 'not'
            notword = stemmer.stem(notword)
            notword = set(map(int,get_word(inv_index, notword)[1].keys()))
            w = set(map(int,get_word(inv_index, w)[1].keys()))
            return sorted(w.difference(notword))

        else: #in case of just and
            w1 = set(map(int,get_word(inv_index,stemmer.stem(re.findall(r'^\w+',query)[0]))[1].keys()))#w1 contains documents of the word before AND
            w2 = set(map(int,get_word(inv_index, stemmer.stem(re.findall(r'and\s(.+)',query)[0]))[1].keys()))#w2 contains documents of the word after  AND
            return sorted(w1.intersection(w2))
        
    if query.find(' or ') != -1:
        w1 = set(map(int,get_word(inv_index,stemmer.stem(re.findall(r'^\w+',query)[0]))[1].keys()))#w1 contains documents of the word before OR 
        w2 = set(map(int,get_word(inv_index,stemmer.stem(re.findall(r'or\s(.+)',query)[0]))[1].keys()))#w2 contains documents of the word after  OR (after stemming)
        return sorted(w1.union(w2))

In [72]:
print(query_parser('Happiness',inv_index))
print('\n')
print(query_parser('Edinburgh AND Scotland',inv_index))
print('\n')
print(query_parser('income OR taxes',inv_index))
print('\n')
print(query_parser('"income taxes"',inv_index))
print('\n')
print(query_parser('#20(income, taxes)',inv_index))
print('\n')
print(query_parser('"middle east" AND peace',inv_index))
print('\n')
print(query_parser('"islam religion"',inv_index))
print('\n')
print(query_parser('"Financial times" AND NOT BBC',inv_index))
print('\n')
print(query_parser('#15(dow,stocks)',inv_index))

Happiness
[58, 136, 137, 196, 264, 290, 341, 372, 3329, 3362, 3443, 3474, 3638, 3730, 3773, 3856, 3864, 3913]


Edinburgh AND Scotland
[351]


income OR taxes
[3, 14, 16, 23, 24, 32, 33, 34, 39, 41, 42, 43, 46, 65, 92, 106, 112, 113, 125, 141, 144, 146, 161, 163, 168, 170, 171, 172, 173, 175, 177, 178, 184, 185, 186, 190, 216, 226, 228, 241, 248, 257, 262, 268, 270, 282, 287, 304, 313, 314, 315, 316, 317, 319, 322, 325, 326, 328, 329, 351, 357, 360, 361, 366, 370, 375, 3324, 3327, 3329, 3331, 3332, 3333, 3336, 3337, 3341, 3342, 3343, 3345, 3347, 3374, 3387, 3388, 3390, 3396, 3397, 3401, 3403, 3404, 3405, 3407, 3409, 3412, 3415, 3417, 3420, 3437, 3439, 3441, 3443, 3444, 3449, 3460, 3469, 3470, 3472, 3473, 3474, 3475, 3480, 3481, 3482, 3490, 3494, 3495, 3498, 3503, 3504, 3511, 3519, 3525, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3535, 3537, 3542, 3543, 3548, 3561, 3562, 3563, 3567, 3569, 3580, 3582, 3586, 3589, 3590, 3591, 3592, 3593, 3595, 3596, 3597, 3599, 3602, 3604, 3606, 3607, 3608

### Read query and write results to file

In [73]:
def read_queries_boolean(file, inv_index):
    with open(file, 'r') as file, open('results.boolean.txt', 'w') as outfile:
        for i,line in enumerate(file):
            data = line.strip()
            results = query_parser(data,inv_index)
            for result in results:
                outfile.write(str(i+1) + ',' + str(result) + '\n')
                
read_queries_boolean('queryprocessed.lab2.txt', inv_index)

Scotland
Window
replacing
condemning
income OR taxes
income AND NOT taxes
"income taxes"
#10(income, taxes)
"middle east" AND peace


## Ranked search

### Create tfidf document index

In [80]:
def ranked_search(query, inv_index, docs):
    print(query)
    queryprocessed = re.findall(r'\w+',query)#all words in query (Tokenization)
    queryprocessed = [stemmer.stem(word) for word in queryprocessed if stemmer.stem(word.lower()) not in STwords]

    query_index = {}
    tfidf_index = {}
    for doc in docs:#for each doc create dict tfidf
        tfidf_index[doc] = {} 

    for word in queryprocessed:
        query_index[word] = 1 + math.log(queryprocessed.count(word),10)
        for doc in docs:
            # Search which documents contain the word and calculate tfidf, if word not in document, then tfidf=0.
                if doc in inv_index[word][1].keys():
                    tfidf_index[doc][word] = (1 + math.log(len(inv_index[word][1][doc]),10)) * (math.log((len(docs)/inv_index[word][0]),10))# 1+ log(tf [t,d])*log(N/df)
                else:
                    tfidf_index[doc][word] = 0       #if word not in document, then tfidf=0.
                    
    scores = []#sum of w(t,d)
    for doc in docs:
        score = 0
        for word in queryprocessed:
            if tfidf_index[doc][word] != 0:
                score += query_index[word] * (tfidf_index[doc][word])
        if score > 0:
            scores.append((doc,score))
    
    return sorted(scores, key=lambda tup: tup[1], reverse=True)

In [75]:
print(ranked_search("income tax reduction", inv_index, docs)[:10])
print('\n')
print(ranked_search("stock market in Japan", inv_index, docs)[:10])
print('\n')
print(ranked_search("health industry", inv_index, docs)[:10])
print('\n')
print(ranked_search("the Robotics industries", inv_index, docs)[:10])
print('\n')
print(ranked_search("the peace process in the middle east", inv_index, docs)[:10])
print('\n')
print(ranked_search("information retrieval", inv_index, docs)[:10])
print('\n')
print(ranked_search("Dow Jones industrial average stocks", inv_index, docs)[:10])
print('\n')
print(ranked_search("will be there a reduction in the income taxes?", inv_index, docs)[:10])
print('\n')
print(ranked_search("the gold prices versus the dollar price", inv_index, docs)[:10])

income tax reduction
[('65', 4.804034014375512), ('3533', 4.7264081984633775), ('3562', 3.5453626928666857), ('3608', 3.4909556861709032), ('141', 3.3261651748396766), ('361', 3.3261651748396766), ('92', 3.2310821930752125), ('3829', 3.1818363683925575), ('3420', 3.1273075274176665), ('3734', 3.05610026861608)]


stock market in Japan
[('3693', 3.7149285503334486), ('3459', 3.4245701312235775), ('287', 3.3848398395205095), ('21', 3.3128376677560882), ('3416', 3.309031248787746), ('3570', 3.177220857140374), ('3797', 3.121412090937466), ('3931', 2.9838837651652943), ('3495', 2.9256011815861958), ('3584', 2.900786909592978)]


health industry
[('252', 3.053321495111291), ('351', 3.002212881653417), ('3648', 2.7095653718849486), ('129', 2.612484631084815), ('3508', 2.4949732706802545), ('3370', 2.4597902931929645), ('101', 2.3704698314537866), ('3443', 2.330305967012219), ('38', 2.217775493561936), ('216', 2.1691837046408216)]


the Robotics industries
[('129', 6.504299858263528), ('135',

### Read query and write results to file

In [76]:
def queries_ranked(file, tfidf_index, vocab):
    res = []
    with open(file, 'r') as file, open('results.ranked.txt', 'w') as outfile:
        for i,line in enumerate(file):
            data = line.strip()
            data = re.sub(r'^\d+\s','',data).strip()
            #ranked_search(query, inv_index, docs)
            results = ranked_search(data,tfidf_index, vocab)
            for j,result in enumerate(results):
                if j < 150:
                    outfile.write(str(i+1) + ',' + str(result[0]) + ',' + str(round(result[1],4)) + '\n')
                else:
                    continue
                        
queries_ranked('queryprocessed.txt', inv_index,docs)

income tax reduction
peace in the Middle East
unemployment rate in UK
industry in scotland
the industries of computers
Microsoft Windows
stock market in Japan
the education with computers
health industry
campaigns of political parties
